In [ ]:
import sys
sys.path.append('../')

In [ ]:
from api.utils.transformer import Transformer
from api.tests.test_parser import get_training_data

In [ ]:
training_data = get_training_data(9999999999)

In [ ]:
transformer = Transformer()
X, y = transformer.fit_transform(training_data)

In [ ]:
X

# BUILD YOUR MODEL

# Save your model

In [ ]:
from api.utils.model import TWCModel

In [ ]:
model = TWCModel()
model.transformer = transformer
model.model = None #YOUR MODEL
model.save('yourmodelpath.p')